In [1]:
suppressPackageStartupMessages(require(tidyverse))
suppressPackageStartupMessages(require(data.table))
library(latex2exp)


Warning message:
“package ‘ggplot2’ was built under R version 3.6.2”Warning message:
“package ‘tibble’ was built under R version 3.6.3”Warning message:
“package ‘tidyr’ was built under R version 3.6.2”Warning message:
“package ‘dplyr’ was built under R version 3.6.2”Warning message:
“package ‘forcats’ was built under R version 3.6.2”

In [2]:
outd <- file.path(getwd(), 'out_v3')

traits <- fread('cascade.input.files.tsv')


In [32]:
imp_var_annot_df <- fread(cmd=paste('zcat', file.path(outd, 'cascade.imp.hits.var.annot.tsv.gz'))) %>%
rename('CHROM'='#CHROM') %>%
select(-LoF, -LoF_filter, -LoF_flags, -LoF_info, -maf)


In [33]:
imp_var_annot_df %>% dim()

[1] 786  10

In [34]:
imp_var_annot_df %>% colnames()

[1] "CHROM"             "POS"               "REF"              
 [4] "ALT"               "ID"                "Gene"             
 [7] "Consequence"       "HGVSp"             "consequence_field"
[10] "Gene_symbol"

In [3]:
imp_anno_df <-
fread(cmd=paste('zstdcat', file.path(outd, 'cascade.imp.hits.tsv.zst'), sep=' '), sep='\t')


In [4]:
imp_anno_df %>% dim()

[1] 776  23

In [5]:
imp_anno_df %>% colnames()

[1] "ID"                "REF"               "ALT"              
 [4] "Effect"            "StdErr"            "Direction"        
 [7] "HetISq"            "HetChiSq"          "HetDf"            
[10] "HetPVal"           "maf"               "MACH_R2"          
[13] "CHROM"             "POS"               "P"                
[16] "CM"                "id"                "prev"             
[19] "block"             "name"              "is_outside_of_MHC"
[22] "is_rare"           "is_autosome"

In [52]:
imp_anno_2_df <- imp_anno_df %>%
rename('ID_old'='ID')%>%
left_join(
    imp_var_annot_df %>%
    select(CHROM, POS, REF, ALT, ID, Gene, Gene_symbol, Consequence, HGVSp) %>%
    unique(), 
    by=c('CHROM', 'POS', 'REF', 'ALT')
)

In [53]:
imp_anno_2_df %>% dim()

[1] 776  28

In [54]:
imp_anno_2_df %>% colnames()

[1] "ID_old"            "REF"               "ALT"              
 [4] "Effect"            "StdErr"            "Direction"        
 [7] "HetISq"            "HetChiSq"          "HetDf"            
[10] "HetPVal"           "maf"               "MACH_R2"          
[13] "CHROM"             "POS"               "P"                
[16] "CM"                "id"                "prev"             
[19] "block"             "name"              "is_outside_of_MHC"
[22] "is_rare"           "is_autosome"       "ID"               
[25] "Gene"              "Gene_symbol"       "Consequence"      
[28] "HGVSp"

In [6]:
array_anno_df <- 
fread(cmd=paste('zstdcat', file.path(outd, 'cascade.array.hits.tsv.zst'), sep=' '), sep='\t')


In [7]:
array_anno_df %>% dim()

[1] 9735   40

In [8]:
array_anno_df %>% colnames()

[1] "ID"                "Effect"            "StdErr"           
 [4] "P"                 "Direction"         "HetISq"           
 [7] "HetChiSq"          "HetDf"             "HetPVal"          
[10] "name"              "CHROM"             "POS"              
[13] "REF"               "ALT"               "Gene"             
[16] "Consequence"       "HGVSp"             "LoF"              
[19] "LoF_filter"        "LoF_flags"         "LoF_info"         
[22] "consequence_field" "f_miss"            "f_miss_bileve"    
[25] "f_miss_wcsg"       "freq"              "hwe_p"            
[28] "maf"               "ld_indep"          "wcsg_only"        
[31] "bileve_only"       "mgi"               "mgi_notes"        
[34] "all_filters"       "Gene_symbol"       "MAF"              
[37] "variant"           "Csq"               "is_outside_of_MHC"
[40] "is_rare"

In [55]:
common_cols <- intersect(names(array_anno_df), names(imp_anno_2_df))
common_cols

[1] "ID"                "Effect"            "StdErr"           
 [4] "P"                 "Direction"         "HetISq"           
 [7] "HetChiSq"          "HetDf"             "HetPVal"          
[10] "name"              "CHROM"             "POS"              
[13] "REF"               "ALT"               "Gene"             
[16] "Consequence"       "HGVSp"             "maf"              
[19] "Gene_symbol"       "is_outside_of_MHC" "is_rare"

In [63]:
imp_anno_2_df %>%
count(Consequence)

Consequence,n
<chr>,<int>
3_prime_UTR_variant,21
5_prime_UTR_variant,9
downstream_gene_variant,37
intergenic_variant,140
intron_variant,460
missense_variant,4
non_coding_transcript_exon_variant,16
regulatory_region_variant,27
splice_region_variant,1


In [64]:
non_coding_df <- array_anno_df %>%
filter(Csq == 'non-coding') %>%
select(all_of(common_cols)) %>%
bind_rows(
    imp_anno_2_df %>% 
    filter(!Consequence %in% c('missense_variant', 'stop_gained', 'synonymous_variant')) %>%
    select(all_of(common_cols))
)


In [65]:
non_coding_df %>% dim()

[1] 9119   21

In [66]:
p.imp <- non_coding_df %>% 
filter(is_outside_of_MHC) %>%
group_by(name) %>%
mutate(
    rank_abs_effect = rank(-abs(Effect))
) %>%
ungroup() %>%
mutate(
    outlier = if_else(abs(Effect) >= .1, TRUE, FALSE),
    plot_label = ID,
    plot_label = str_replace(plot_label, '[(][)]$', ''),
    plot_label = str_replace(plot_label, '[(]NA[)]$', ''),
    plot_label = str_replace(plot_label, '_', ':'),    
    ggrepel = if_else((abs(Effect) >= .1) & (rank_abs_effect < 5), plot_label, "")
) %>%
ggplot(
    aes(x=maf, y=Effect, color=outlier, label = ggrepel)
) +
geom_hline(yintercept = 0, linetype="dashed") +
geom_point(alpha=.5) + theme_bw() + 
labs(
    title = TeX('Non-coding variant associations ($p \\,<\\, 5x10^{-9}$)'),
    x = 'Minor allele frequency (log-scale)',
    y = 'BETAs'
) +
theme(
    strip.text = element_text(size=7),
    legend.position="none"
)+
scale_color_brewer(palette="Dark2") + 
# scale_x_continuous(trans="log10", breaks=10 ** c(-2, -1, 0)) + 
scale_x_continuous(trans="log10", breaks=c(0.001, 0.01, 0.1)) + 
facet_wrap(~name, ncol = 6, strip.position = "bottom", labeller = label_wrap_gen(17)) + 
ggrepel::geom_text_repel(size=1.5)


In [67]:
for(ext in c('png', 'pdf')){
    ggsave(file.path(outd, sprintf("cascade.imp.%s", ext)), p.imp, width = 12, height = 14)
}


## prepare supplementary table files
- reformat the tables

In [68]:
supl_tbl_rename_list <- list(
    'BETA'='Effect', 'SE'='StdErr', 'trait'='name', 'MAF'='maf'
)

supl_tbl_cols <- c(
    'CHROM', 'POS', 'REF', 'ALT', 'ID', 'variant', 
    'trait', 'BETA', 'SE', 'P',
    'Direction', 'HetISq', 'HetChiSq', 'HetDf', 'HetPVal',
    'MAF', 'Consequence', 'Gene', 'Gene_symbol', 'HGVSp',
    'is_outside_of_MHC', 'Comments'
)


In [69]:
non_coding_df %>%
mutate(Comments='', P=sprintf('%.2e', P), variant=paste(CHROM, POS, REF, ALT, sep=':'))%>%
rename(unlist(supl_tbl_rename_list)) %>%
select(supl_tbl_cols) %>%
arrange(CHROM, POS, trait) %>% 
rename('#CHROM' = 'CHROM') %>%
fwrite(
    file.path(outd, "cascade.nc.tsv"),
    sep='\t', na = "NA", quote=F
)
